# Import

In [1]:
# System
import os
import time
import json
os.environ["OMP_NUM_THREADS"] = "4" # export OMP_NUM_THREADS=4
os.environ["OPENBLAS_NUM_THREADS"] = "4" # export OPENBLAS_NUM_THREADS=4 
os.environ["MKL_NUM_THREADS"] = "6" # export MKL_NUM_THREADS=6
os.environ["VECLIB_MAXIMUM_THREADS"] = "4" # export VECLIB_MAXIMUM_THREADS=4
os.environ["NUMEXPR_NUM_THREADS"] = "6" # export NUMEXPR_NUM_THREADS=6

# Data processing
import numpy as np
import netCDF4 as nc

# Config

In [2]:
# Config path
root = '/Volumes/Expansion/User_Backup/b08209033/111-2_IVT_analysis/'
folder = '2023_0330'
file = 'src/config.json'
config_path = os.path.join(os.path.join(root, folder), file)

# Import config
with open(config_path) as infile:
    config = json.load(infile)
    infile.close()

# Update config
    # IVT variable
config.update({"IVTPath": os.path.join(config["DataPath"], "IVT")})
config.update({"IVT_fname": "IVT_raw.npz"})
#config.update({"IVT_is_filtered": True})

# Export config
with open(config_path, 'w') as outfile:
    json.dump(config, outfile, sort_keys=True)
    outfile.close()

# Self-defiend Objects

## IVT

In [3]:
# Input .nc object
# Return 
    # Lat
    # Lon
    # Original IVT [year, day, lat, lon]
    # seasonal IVT [day, lat, lon] (optional)
def getIVT(rootgrp, seasonal_cycle = False):
    # Determine array shape
        # Find region index
    lon = rootgrp['lon'][:]
    lat = rootgrp['lat'][:]
        # Find level index
    level = rootgrp['plev'][:]
        # Calculate thickness
    thickness = (level[:-1] - level[1:])
    
    # Original file
        # Read variables
    u = rootgrp['u'][:,:,:,:].reshape(-1, 365, level.shape[0], lat.shape[0], lon.shape[0])
    v = rootgrp['v'][:,:,:,:].reshape(-1, 365, level.shape[0], lat.shape[0], lon.shape[0])
    q = rootgrp['q'][:,:,:,:].reshape(-1, 365, level.shape[0], lat.shape[0], lon.shape[0])
        # Calculate IVT
    interp_u = (u[:,:,1:,:,:] + u[:,:,:-1,:,:])/2
    interp_v = (v[:,:,1:,:,:] + v[:,:,:-1,:,:])/2
    interp_q = (q[:,:,1:,:,:] + q[:,:,:-1,:,:])/2
        # Broadcast 
    broadcast_thickness = thickness[np.newaxis, np.newaxis, :, np.newaxis, np.newaxis]      
        # Integrate over levels
    IVT_x = np.sum((interp_u * interp_q * broadcast_thickness)/9.81, axis = 2)
    IVT_y = np.sum((interp_v * interp_q * broadcast_thickness)/9.81, axis = 2)
    
    # Trend file
    if (seasonal_cycle != False):
            # Read variables
        u = seasonal_cycle['u'][:,:,:,:]
        v = seasonal_cycle['v'][:,:,:,:]
        q = seasonal_cycle['q'][:,:,:,:]
            # Calculate IVT
        interp_u = (u[:,1:,:,:] + u[:,:-1,:,:])/2
        interp_v = (v[:,1:,:,:] + v[:,:-1,:,:])/2
        interp_q = (q[:,1:,:,:] + q[:,:-1,:,:])/2
            # Broadcast 
        broadcast_thickness = thickness[np.newaxis, :, np.newaxis, np.newaxis]
            # Integrate over levels
        IVT_x_trend = np.sum((interp_u * interp_q * broadcast_thickness)/9.81, axis = 1)
        IVT_y_trend = np.sum((interp_v * interp_q * broadcast_thickness)/9.81, axis = 1)
        return lat, lon, np.array((IVT_x, IVT_y)), np.array((IVT_x_trend, IVT_y_trend))
    return lat, lon, np.array((IVT_x, IVT_y))

# Data preprocessing

In [4]:
# Process
count = time.time()
os.chdir(config["DataPath"])
rootgrp = nc.Dataset("dataset.nc")
trend = nc.Dataset("seasonal_cycle.nc")
lat, lon, IVT, IVT_trend = getIVT(rootgrp, trend)
rootgrp.close()
trend.close()
print(f"Takes {(time.time()-count):.3f} sec")

Takes 67.383 sec


In [5]:
os.chdir(config["IVTPath"])
np.savez(config["IVT_fname"], IVT_original = IVT, IVT_seasonal = IVT_trend)

# Undetermined blocks

# Unused blocks